In [34]:
!pip install fastapi uvicorn nest-asyncio pyngrok python-multipart pymupdf


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-24' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

In [35]:
pip install python-docx


In [36]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import PlainTextResponse
import fitz  # PyMuPDF

import nest_asyncio
from pyngrok import ngrok
import uvicorn
from docx import Document
import io


In [37]:
from transformers import pipeline
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


In [46]:
!ngrok config add-authtoken "Paste your ngrok authtoken here"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [39]:

def extract_text_from_file(contents, filename):

    extension = filename.lower().split('.')[-1]

    if extension == 'pdf':
        doc = fitz.open(stream=contents, filetype='pdf')
        text = ""
        for page in doc:
            text += page.get_text()
        return text

    elif extension == 'txt':
        return contents.decode('utf-8')  # assuming contents is bytes

    elif extension == 'docx':
        doc = Document(io.BytesIO(contents))
        text = '\n'.join([p.text for p in doc.paragraphs])
        return text

    else:
        raise ValueError("Unsupported file type")


In [40]:
import spacy

nlp = spacy.load("en_core_web_sm")  # Load English model

def chunk_by_sentences_spacy(text, max_chunk_size=1000):
    doc = nlp(text)
    chunks = []
    current_chunk = ""

    for sent in doc.sents:
        sentence = sent.text.strip()
        # Check if adding this sentence exceeds max chunk size
        if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:
            current_chunk += (" " + sentence) if current_chunk else sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence

    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# Test
text = "Ali is learning NLP. He is working on summarization! This project is interesting? Yes, it really is."
chunks = chunk_by_sentences_spacy(text, max_chunk_size=50)
print(chunks)


['Ali is learning NLP.', 'He is working on summarization!', 'This project is interesting? Yes, it really is.']


In [41]:
text = "Ali is learning NLP. He is working on summarization. This project is interesting!"
chunks = chunk_by_sentences(text, max_chunk_size=50)
print(chunks)


['Ali is learning NLP.', 'He is working on summarization.', 'This project is interesting!']


In [42]:
import textwrap

In [43]:
app = FastAPI()

@app.post("/upload", response_class=PlainTextResponse)
async def upload_pdf(file: UploadFile = File(...)):
    contents = await file.read()
    text = extract_text_from_file(contents=contents, filename=file.filename)

    print("text is read from pdf ")
    chunks = chunk_by_sentences(text=text)
    print("chunks are made from text")
    # Summarize each chunk
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
        summaries.append(summary)
    print("summaries is generated")
    print(summaries)
    return "\n".join(summaries)


In [44]:
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://127.0.0.1:5500", "http://localhost:5000"],  # add all your frontend origins here
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


In [45]:
# Setup for running inside Jupyter/Colab
nest_asyncio.apply()
ngrok.kill()
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Run API
uvicorn.run(app, port=8000)

Public URL: NgrokTunnel: "https://5c1c7e8467c0.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [243]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


text is read from pdf 
chunks are made from text


Your max_length is set to 130, but your input_length is only 124. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=62)


summaries is generated
[" Pakistan, officially the Islamic Republic of Pakistan, is the fifth-most populous country in South Asia . Islamabad is the nation's capital, while Karachi is its largest city and financial centre . Pakistan is the 33rd-largest country by area .", " Pakistan is the site of several ancient cultures, including the 8,500-year-old Neolithic site of Mehrgarh in Balochistan . It shares a maritime border with Oman in the Gulf of Oman and is separated from Tajikistan in the northwest by Afghanistan's narrow Wakhan Corridor . Pakistan gained independence in 1947 after the partition of the British Indian Empire .", " In 1971, the exclave of East Pakistan seceded as the new country of Bangladesh after a nine-month-long civil war . Pakistan's political history has been characterized by periods of significant economic and military growth as well as those of political and economic instability . The country continues to face challenges, including poverty, illiteracy, corrupti

Your max_length is set to 130, but your input_length is only 22. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


summaries is generated
[' The Himalayas are home to Mount Everest, the tallest peak on Earth . The world is filled with an incredible variety of landscapes, cultures, and natural wonders . Exploring beautiful places around the globe offers a feast for the eyes and the soul .', ' The Rocky Mountains stretch from Canada to the United States . The Alps are renowned for their picturesque villages, skiing resorts, and lush valleys dotted with wildflowers during the summer .', ' The Maldives, an archipelago of over a thousand coral islands, is a paradise for beach lovers and honeymooners . The Rocky Mountain National Park in Colorado is a popular destination, showcasing dramatic cliffs, alpine lakes, and vast forests .', ' The Caribbean is famous for its vibrant culture, reggae music, and lively festivals . The Great Barrier Reef in Australia is the largest coral reef system in the world . The western coast of the United States offers dramatic cliffs and expansive beaches .', ' The city of R

Your max_length is set to 130, but your input_length is only 115. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)
Your max_length is set to 130, but your input_length is only 95. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)


summaries is generated
[' Google hereby grants permission to produce the tables and figures in this paper solely for use in journalistic or scholarly works . We propose a new simple network architecture, the Transformer, based solely on attention mechanisms .', ' Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train . We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing with large and limited training data .', ' Ashish, with Illia, designed and implemented the first Transformer models and has been crucially involved in every aspect of this work . Niki designed, implemented, tuned and evaluated countless model variants in our original codebase . Llion also experimented with novel model variants, was responsible for our initial codebase, and was responsible . Noam proposed scaled dot-product attention, multi-

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [243]
